In [40]:
import os
import cv2
from ultralytics import YOLO
import xml.etree.ElementTree as ET
from torchvision import transforms
import re
import torch.optim as optim

In [43]:
def extract_plate_images(yolo_model, image_dir, xml_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for image_file in os.listdir(image_dir):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_dir, image_file)
            image = cv2.imread(image_path)

            # Run YOLOv8 inference to detect plates
            results = yolo_model(image_path)
            for result in results:
                boxes = result.boxes.xyxy.cpu().numpy()  # Extract bounding box coordinates

                for i, box in enumerate(boxes):
                    x1, y1, x2, y2 = map(int, box[:4])  # Get bounding box coordinates
                    plate_img = image[y1:y2, x1:x2]  # Crop the plate region

                    # Save the cropped plate image
                    output_file = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_plate{i}.jpg")
                    cv2.imwrite(output_file, plate_img)

# Example usage
image_dir = 'images/train'  # Directory where your images are stored
xml_dir = 'labels/train/'  # Directory where your XML files are stored (if needed)
output_dir = 'cropped_plates/'  # Directory to save cropped plates

yolo_model = YOLO('runs\\train\\alpr_model\\weights\\best.pt')
extract_plate_images(yolo_model, image_dir, xml_dir, output_dir)



image 1/1 c:\Users\Alvino Angelo\Documents\UMD\DATA612\ALPR\images\train\1.E 3977 QM-09-19.jpeg: 256x640 1 plate, 6.4ms
Speed: 1.0ms preprocess, 6.4ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

image 1/1 c:\Users\Alvino Angelo\Documents\UMD\DATA612\ALPR\images\train\10.E 5894 SZ-09-17.jpg: 288x640 1 plate, 7.1ms
Speed: 1.0ms preprocess, 7.1ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 c:\Users\Alvino Angelo\Documents\UMD\DATA612\ALPR\images\train\100.E 6467 QW-09-20.jpeg: 256x640 1 plate, 6.4ms
Speed: 0.0ms preprocess, 6.4ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 640)

image 1/1 c:\Users\Alvino Angelo\Documents\UMD\DATA612\ALPR\images\train\101.E 4604 PS-09-17.jpeg: 256x640 1 plate, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

image 1/1 c:\Users\Alvino Angelo\Documents\UMD\DATA612\ALPR\images\train\102.E 6984 P-07-20.jpeg: 352x640 1 plate, 6.5ms
Speed:

In [229]:
import PIL
import torchvision.transforms as transforms
import os

# Directory paths
input_dir = 'CRNN/images/test'
output_dir = 'test_preprocessed_plates'

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define preprocessing function
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img = img.resize((100, 32), PIL.Image.LANCZOS)  # Resize the image
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    img = transform(img)
    return img


In [233]:
for image_name in os.listdir(input_dir):
    image_path = os.path.join(input_dir, image_name)
    
    if os.path.isfile(image_path) and image_name.endswith(('.jpg', '.jpeg', '.png')):
        # Preprocess image
        preprocessed_img = preprocess_image(image_path)
        
        # Convert tensor back to image format and save
        save_path = os.path.join(output_dir, image_name)
        preprocessed_img = transforms.ToPILImage()(preprocessed_img.squeeze(0))
        preprocessed_img.save(save_path)

print("Preprocessing complete. Images saved to:", output_dir)


Preprocessing complete. Images saved to: test_preprocessed_plates


In [259]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CRNN(nn.Module):
    def __init__(self, imgH, nc, nclass, nh):
        super(CRNN, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(nc, 64, kernel_size=3, stride=1, padding=1),  # Conv1
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # Pool1 -> Output: (64, 16, imgW/2)

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # Conv2
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # Pool2 -> Output: (128, 8, imgW/4)

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  # Conv3
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # Pool3 -> Output: (256, 4, imgW/8)

            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),  # Conv4
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # Pool4 -> Output: (256, 2, imgW/16)

            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),  # Conv5
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.MaxPool2d((2, 2), (2, 2))  # Pool5 -> Output: (512, 1, imgW/32)
        )

        self.rnn = nn.Sequential(
            nn.LSTM(512, nh, bidirectional=True, batch_first=True),  # LSTM1
            nn.Linear(nh * 2, nh),  # Bidirectional LSTM outputs * 2
            nn.LSTM(nh, nh, bidirectional=True, batch_first=True),  # LSTM2
            nn.Linear(nh * 2, nclass)  # Output layer
        )

    def forward(self, x):
        conv = self.cnn(x)
        b, c, h, w = conv.size()
        print(f"Feature map size after conv layers: {h}x{w}")
        assert h == 1, f"The height of the feature map must be 1, but got {h}"
        conv = conv.squeeze(2)  # Remove the height dimension
        conv = conv.permute(0, 2, 1)  # [batch, width, channels]

        # Run through the first LSTM layer
        rnn_out, _ = self.rnn[0](conv)  # Only take the output, discard hidden states
        
        # Pass through the first Linear layer
        rnn_out = self.rnn[1](rnn_out)
        
        # Run through the second LSTM layer
        rnn_out, _ = self.rnn[2](rnn_out)  # Only take the output, discard hidden states
        
        # Pass through the final Linear layer
        rnn_out = self.rnn[3](rnn_out)
        
        return rnn_out



# Initialize the CRNN model
nclass = 36  # 10 digits + 26 uppercase letters
crnn_model = CRNN(imgH=32, nc=1, nclass=nclass, nh=256)


In [260]:
def extract_label_and_filename(filename):
    # Extract the base filename without the extension
    base_filename = os.path.splitext(filename)[0]
    
    # Extract the portion of the filename before the first hyphen
    match = re.search(r'\.(.*?)\-', base_filename)
    if match:
        label = match.group(1)
    else:
        label = base_filename  # If no hyphen, use the whole base filename
    
    # Add back the .jpg extension to the filename
    label = label.replace(" ", "")
    full_filename = base_filename + ".jpg"
    
    return label, full_filename

def process_jpg_folder(folder_path):
    labels = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
            label, full_filename = extract_label_and_filename(file_name)
            labels.append({"filename": full_filename, "label": label})
    
    return labels


In [261]:
folder_path = 'preprocessed_plates'  # Replace with your actual folder path
labels = process_jpg_folder(folder_path)
print(labels)

[{'filename': '1.E 3977 QM-09-19_plate0.jpg', 'label': 'E3977QM'}, {'filename': '10.E 5894 SZ-09-17_plate0.jpg', 'label': 'E5894SZ'}, {'filename': '100.E 6467 QW-09-20_plate0.jpg', 'label': 'E6467QW'}, {'filename': '101.E 4604 PS-09-17_plate0.jpg', 'label': 'E4604PS'}, {'filename': '102.E 6984 P-07-20_plate0.jpg', 'label': 'E6984P'}, {'filename': '103.E 6387 PL-04-20_plate0.jpg', 'label': 'E6387PL'}, {'filename': '104.E 4939 PAE-07-21_plate0.jpg', 'label': 'E4939PAE'}, {'filename': '105.E 6430 QO-12-19_plate0.jpg', 'label': 'E6430QO'}, {'filename': '106.E 4670 TL-05-22_plate0.jpg', 'label': 'E4670TL'}, {'filename': '107.E 4145 QZ-12-20_plate0.jpg', 'label': 'E4145QZ'}, {'filename': '108.E 5860 ST-03-21_plate0.jpg', 'label': 'E5860ST'}, {'filename': '109.E 4026 SL-02-20_plate0.jpg', 'label': 'E4026SL'}, {'filename': '11.E 6562 QO-12-19_plate0.jpg', 'label': 'E6562QO'}, {'filename': '110.E 4753 QV-08-20_plate0.jpg', 'label': 'E4753QV'}, {'filename': '111.E 6810 IX-05-20_plate0.jpg', 'lab

In [262]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class PlateDataset(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        self.image_dir = image_dir
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.labels[idx]['filename'])
        image = Image.open(image_path).convert('L')  # Convert to grayscale
        label = self.labels[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, label

# Transformation
transform = transforms.Compose([
    transforms.Resize((32, 512)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

labels = process_jpg_folder('preprocessed_plates')

dataset = PlateDataset(image_dir='preprocessed_plates', labels=labels, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [263]:
# Define optimizer and CTC loss
optimizer = optim.Adam(crnn_model.parameters(), lr=0.0001)
ctc_loss = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)

In [264]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    crnn_model.train()
    epoch_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        # Forward pass through the CRNN model
        preds = crnn_model(images)  # preds should be [batch_size, seq_len, num_classes]

        # Reshape preds to match the expected shape (T, N, C)
        preds = preds.permute(1, 0, 2)  # Now preds is [seq_len, batch_size, num_classes]

        # Prepare CTC loss inputs
        preds_size = torch.IntTensor([preds.size(0)] * preds.size(1))  # [batch_size] -> [T, T, T, T]
        label_lengths = torch.IntTensor([len(label) for label in labels])  # [batch_size]

        # Flatten the label sequences into a single tensor
        flattened_labels = []
        for label in labels:
            flattened_labels.extend([ord(c) - ord('A') for c in label])
        labels_tensor = torch.IntTensor(flattened_labels)  # 1D tensor of all labels

        # Check dimensions for debugging
        print(f"Batch {batch_idx + 1} - preds shape: {preds.shape}, preds_size: {preds_size}, label_lengths: {label_lengths.shape}, labels_tensor: {labels_tensor.shape}")

        # Calculate CTC loss
        loss = ctc_loss(preds, labels_tensor, preds_size, label_lengths)
        epoch_loss += loss.item()

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress
        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Print epoch loss
    print(f"Epoch {epoch+1} Loss: {epoch_loss / len(train_loader):.4f}")

# Save the trained model
torch.save(crnn_model.state_dict(), 'crnn_model.pth')


Feature map size after conv layers: 1x16
Batch 1 - preds shape: torch.Size([16, 32, 36]), preds_size: tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16], dtype=torch.int32), label_lengths: torch.Size([32]), labels_tensor: torch.Size([230])
Epoch [1/10], Batch [1/11], Loss: -1.8224
Feature map size after conv layers: 1x16
Batch 2 - preds shape: torch.Size([16, 32, 36]), preds_size: tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16], dtype=torch.int32), label_lengths: torch.Size([32]), labels_tensor: torch.Size([231])
Feature map size after conv layers: 1x16
Batch 3 - preds shape: torch.Size([16, 32, 36]), preds_size: tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16], dtype=torch.int32), label_lengths: torch.Size([32]), labels_tensor: t

In [265]:
# Assuming the CRNN model class is defined as before
def load_model(model_path, imgH=32, nc=1, nclass=36, nh=256):
    model = CRNN(imgH=imgH, nc=nc, nclass=nclass, nh=nh)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
    model.eval()  # Set the model to evaluation mode
    return model

model_path = 'crnn_model.pth'  # Path to the trained model
model = load_model(model_path)


In [266]:
def extract_label_and_filenames(filename):
    # Extract the base filename without the extension
    base_filename = os.path.splitext(filename)[0]
    
    # Extract the portion of the filename before the first hyphen
    match = re.search(r'\.(.*?)\-', base_filename)
    if match:
        label = match.group(1)
    else:
        label = base_filename  # If no hyphen, use the whole base filename
    label = label.replace(" ", "")
    # Add back the .jpg extension to the filename
    full_filename = base_filename + ".jpeg"
    
    return label, full_filename

def process_jpeg_folder(folder_path):
    labels = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
            label, full_filename = extract_label_and_filenames(file_name)
            labels.append({"filename": full_filename, "label": label})
    
    return labels


In [267]:
# Assuming you have a test dataset similar to your training dataset
test_labels = process_jpeg_folder('test_preprocessed_plates')

test_image_dir = 'test_preprocessed_plates'  # Replace with the actual test image directory
test_dataset = PlateDataset(image_dir=test_image_dir, labels=test_labels, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [235]:
print(test_labels)

[{'filename': '351.E 6730 RC-07-19.jpeg', 'label': 'E6730RC'}, {'filename': '352.E 5053 RG-09-20.jpeg', 'label': 'E5053RG'}, {'filename': '353.E 6270 SM-06-20.jpeg', 'label': 'E6270SM'}, {'filename': '354.E 6250 PAJ-02-22.jpeg', 'label': 'E6250PAJ'}, {'filename': '355.E 6547 PAF-09-21.jpeg', 'label': 'E6547PAF'}, {'filename': '356.E 6225 SJ-11-19.jpeg', 'label': 'E6225SJ'}, {'filename': '357.E 4679 PAD-06-21.jpeg', 'label': 'E4679PAD'}, {'filename': '358.E 3438 SJ-10-19.jpeg', 'label': 'E3438SJ'}, {'filename': '359.E 6270 OZ-01-21.jpeg', 'label': 'E6270OZ'}, {'filename': '360.E 4690 PAL-04-22.jpeg', 'label': 'E4690PAL'}, {'filename': '361.E 3193 PS-08-17.jpeg', 'label': 'E3193PS'}, {'filename': '362.E 6525 SF-09-21.jpeg', 'label': 'E6525SF'}, {'filename': '363.E 6263 RU-07-20.jpeg', 'label': 'E6263RU'}, {'filename': '364.E 6528 P-11-18.jpeg', 'label': 'E6528P'}]


In [268]:
def decode_predictions(preds, labels_map):
    preds = preds.permute(1, 0, 2)  # Change to [batch, width, channels]
    preds = nn.functional.log_softmax(preds, dim=2)
    preds = torch.argmax(preds, dim=2)
    preds = preds.transpose(1, 0).contiguous().view(-1)

    pred_texts = []
    for pred in preds:
        if pred != 0:  # Ignore the 'blank' label
            pred_texts.append(labels_map[pred.item()])

    return ''.join(pred_texts)

# Define a character map (A-Z, 0-9, blank)
labels_map = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

In [271]:
def evaluate_model(model, test_loader, labels_map):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            preds = model(images)
            pred_text = decode_predictions(preds, labels_map)

            # Compare the prediction with the ground truth
            ground_truth = labels[0]  # Since batch_size is 1
            if pred_text == ground_truth:
                correct += 1
            total += 1
            print(f"Predicted: {pred_text}, Ground Truth: {ground_truth}")

    accuracy = correct / total * 100
    print(f"Accuracy: {accuracy:.2f}%")

# Run the evaluation
evaluate_model(model, test_loader, labels_map)


Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E6730RC
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E5053RG
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E6270SM
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E6250PAJ
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E6547PAF
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E6225SJ
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E4679PAD
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E3438SJ
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E6270OZ
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E4690PAL
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E3193PS
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E6525SF
Feature map size after conv layers: 1x16
Predicted: , Ground Truth: E6263RU
Feature 